# <strong><u>2: Speed and Upsample Notebook |</u></strong>

This notebook is designed for speed alignment and data up-sampling in gesture analysis. It uses Python libraries like Pandas for data management, NumPy for numerical operations, and Matplotlib for plotting.

In this workflow, the notebook reads in keypoint data from a pickled DataFrame and performs operations to align speed and upsample the data.

### <strong>Requirements</strong>

To run this notebook, you will need the following Python packages:

- pandas
- numpy
- scipy
- matplotlib

You can install these packages using pip:
```shell
    pip install pandas numpy scipy matplotlib
```

### <strong>Key Variables and Functions</strong>
- `KEYPOINT`: Variable specifying which keypoint (e.g., "right_wrist") to focus on
- `keypoints_df`: DataFrame storing keypoint data



### <strong>Importing Libraries</strong>
- `pandas`: For DataFrame operations and data analysis
- `numpy`: For numerical computations
- `scipy`: For scientific computations, including filtering
- `matplotlib`: For plotting and visualizations


In [ ]:
import pandas as pd
import numpy as np
from scipy.signal import savgol_filter
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

### <strong>Choosing The Right Keypoints</strong>

Using this script, we can isolate the keypoints with which we would like to perform the analyses on. Below is our index of Keypoints as `keypoint_name(keypoint_id)`:

![Keypoints](https://i.imgur.com/SkXsWRZ.png)

### <strong><u>Available Keypoints:</u></strong>
<ul>
    <li><font color='#f2ce72'>nose</font>
    <li><font color='#f2ce72'>right_eye</font>
    <li><font color='#f2ce72'>left_eye</font>
    <li><font color='#f2ce72'>right_shoulder</font>
    <li><font color='#f2ce72'>left_shoulder</font>
    <li><font color='#f2ce72'>right_elbow</font>
    <li><font color='#f2ce72'>left_elbow</font>
    <li><font color='#f2ce72'>right_wrist</font>
    <li><font color='#f2ce72'>left_wrist</font>
</ul>


In [ ]:
PARTICIPANT = "TEST_VIDEO"
KEYPOINTS = ["right_wrist", "right_index"]

### <strong>Load Data</strong>
In this section, the data is loaded into the notebook for further processing. You can adjust the source file paths as needed.

In [ ]:
# Load keypoint data
keypoints_df = pd.read_csv(f'C:/Users/cosmo/OneDrive/Desktop/Repositories/Co-Speech-Gesture-Automation/TEST_FILES/' + PARTICIPANT + '_keypoints.csv', encoding='utf-8')
column_names = [
    "time_ms", 
    "right_shoulder_x", "right_shoulder_y", 
    "left_shoulder_x", "left_shoulder_y", 
    "right_elbow_x", "right_elbow_y", 
    "left_elbow_x", "left_elbow_y", 
    "right_wrist_x", "right_wrist_y", 
    "left_wrist_x", "left_wrist_y", 
    "right_eye_x", "right_eye_y", 
    "left_eye_x", "left_eye_y",
    "nose_x", "nose_y",
    "right_index_x", "right_index_y"
]

keypoints_df.columns = column_names

keypoints_df

### <strong>Calculate Unsmoothed Speed</strong>
Here, the speed of the right wrist is calculated without any smoothing. Parameters such as the sampling rate can be adjusted.

### <strong>Apply Savitzky-Golay Filtering</strong>
The Savitzky-Golay filtering method is applied to smooth the speed data. You can adjust the window size and polynomial order.

This code block is used to smooth the values in the `speed_unsmooth` column of a Pandas DataFrame called `keypoints_df`. The first line of code sets any values in the `speed_unsmooth` column that are below the 20th percentile to 0, effectively removing any low-speed outliers from the data. The second line of code applies a Savitzky-Golay filter to the `speed_unsmooth` column with a window size of 9 and a polynomial order of 2, and stores the smoothed values in a new column called `speed_smooth`.

The `np.percentile()` function is used to calculate the 20th percentile of the `speed_unsmooth` column, which is used as the threshold for removing low-speed outliers. The `savgol_filter()` function is used to apply a Savitzky-Golay filter to the `speed_unsmooth` column, which is a type of smoothing filter that can be used to remove noise from data while preserving important features such as peaks and valleys.

Overall, this code block is used to smooth the values in the `speed_unsmooth` column of a DataFrame using a Savitzky-Golay filter, and remove low-speed outliers from the data. Possible ways to improve the code include adding comments to explain the purpose of each line of code, and using more descriptive variable names.

In [ ]:
# Initialize an empty DataFrame to store final results
final_df = pd.DataFrame()

# Generate the new time series at 10 ms intervals
new_time_series = np.arange(keypoints_df['time_ms'].iloc[0], keypoints_df['time_ms'].iloc[-1], 10)

# Loop through each keypoint specified in the KEYPOINTS list
for keypoint in KEYPOINTS:
    # Extracting original x and y coordinates
    x_original = keypoints_df[f'{keypoint}_x'].values
    y_original = keypoints_df[f'{keypoint}_y'].values
    time_original = keypoints_df['time_ms'].values

    # Interpolation for upsampling to 10 ms intervals
    interp_x = interp1d(time_original, x_original, kind='linear')
    interp_y = interp1d(time_original, y_original, kind='linear')
    x_upsampled = interp_x(new_time_series)
    y_upsampled = interp_y(new_time_series)

    # Smoothing using Savitzky-Golay filter
    window_length = 11  # Choose an appropriate window length
    polyorder = 3  # Choose an appropriate polynomial order
    x_smooth = savgol_filter(x_upsampled, window_length, polyorder)
    y_smooth = savgol_filter(y_upsampled, window_length, polyorder)

    # Speed calculation
    delta_time = np.diff(new_time_series)
    delta_x = np.diff(x_smooth) / delta_time
    delta_y = np.diff(y_smooth) / delta_time
    speed = np.sqrt(np.nan_to_num(delta_x ** 2 + delta_y ** 2) / delta_time)
    
    # Normalize speed to make it comparable across participants
    speed = speed / np.nanmax(speed)
    
    # if speed is below the 20th percentile, set it to 0
    speed[speed < np.nanpercentile(speed, 50)] = 0
    
    # smooth the speed
    speed = savgol_filter(speed, 11, 3)

    # Add to the final DataFrame
    temp_df = pd.DataFrame({
        'time_ms': new_time_series[:-1],
        f'{keypoint}_x': x_smooth[:-1],
        f'{keypoint}_y': y_smooth[:-1],
        f'{keypoint}_speed': speed,
        f'{keypoint}_delta_x': delta_x,
        f'{keypoint}_delta_y': delta_y, 
    })
    if final_df.empty:
        final_df = temp_df
    else:
        final_df = pd.merge(final_df, temp_df, on='time_ms', how='right')

final_df.head()

In [ ]:
print("Length of x_upsampled:", len(x_upsampled))
print("Length of y_upsampled:", len(y_upsampled))
print(len(new_time_series))

### <strong>Save Aligned Data as CSV</strong>
Finally, the aligned data is saved as a CSV file. You can specify the destination path for the output file.

In [ ]:
# Save the aligned data as a CSV file
final_df.to_csv(f'C:/Users/cosmo/OneDrive/Desktop/Repositories/Co-Speech-Gesture-Automation/TEST_FILES/'+ PARTICIPANT + '_processed_data.csv', index=False)
final_df.head()

print(f'File saved as {PARTICIPANT}_processed_data.csv')

### <strong>Plot Data</strong>

The `plot()` function is used to plot the unsmoothed and smoothed speed over time. The `label` parameter is used to set the legend label for each line. The `color` parameter is used to set the color of each line.



In [ ]:
import plotly.graph_objects as go

# Create traces
trace0 = go.Scatter(x=final_df['time_ms'], y=final_df['right_wrist_speed'], mode='lines', name='Right Wrist Speed')
trace1 = go.Scatter(x=final_df['time_ms'], y=final_df['right_wrist_delta_x'], mode='lines', name='Right Wrist Raw Velocity X', yaxis='y2')
trace2 = go.Scatter(x=final_df['time_ms'], y=final_df['right_wrist_delta_y'], mode='lines', name='Right Wrist Raw Velocity Y', yaxis='y2')


# Create layout
layout = go.Layout(title='Right Wrist Speed and Velocity', xaxis=dict(title='Time (ms)'), yaxis1=dict(title='Speed', overlaying='y', side='left'))

# Create figure
fig = go.Figure(data=[trace0], layout=layout)

# Show figure

fig.show()



In [ ]:
# !pip install nbformat==4.2.0